In [3]:
import pandas as pd

In [4]:
import numpy as np

In [5]:
import matplotlib

In [4]:
pip install datetime

Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
from datetime import datetime, timedelta

In [7]:
# -----------------------------
# USER INPUTS
# -----------------------------

#SHEET_NAME = 0                 # or sheet name
LOG_DATE = "2025-07-01"        # date of the log sheet

LOG_DATE = "2025-07-01"  # date of the log sheet

# Full list of parameters
DESCRIPTION = {
    "COMPRESSOR INLET PRESSURE": "CIP",
    "COMPRESSOR INLET AIR TEMP": "CIT",
    "COMPRESSOR DISCHARGE PRESSURE": "CDP",
    "COMPRESSOR DISCHARGE TEMPERATURE": "CDT",
    "IGV POSITION": "IGV_Position",
    "FILTER DIFFERENTIAL PRESSURE": "Filter_DP",
    "GT SPEED": "GT_Speed",
    "MAXIMUM VIBRATION": "VIB",
    "LOAD": "Load"
    
}

df_raw = pd.read_csv(r'GT1 Turbine Data Log Sheet for July 2025.csv', header=2)
df_raw.head(3)

,SUB SYSTEM,DESCRIPTION,KKS,UNIT,RANGE,ALARM,TRIP,0:00,1:00,2:00,...,14:00,15:00,16:00,17:00,18:00,19:00,20:00,21:00,22:00,23:00
0,NaN,GT STATUS,Presel / Base,NaN,NaN,NaN,NaN,PS,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,LOAD,DWATT,MW,0 < x < 126.1,126.0,135.0,95.1,99.0,99.1,...,107.7,105.5,107.8,107.8,105.1,108.6,109.9,98.7,109.6,103.4
2,NaN,REACTIVE POWER,NaN,MVAR,#NAME?,-40.0,-48.0,-13,-17.1,-19.5,...,-7.7,-6.4,-1.5,-1.4,-0.1,-4.0,-7.4,-4.0,-9.0,-7.2


#ef

In [8]:

# Normalize DESCRIPTION column (strip spaces, uppercase)
df_raw["DESCRIPTION"] = df_raw["DESCRIPTION"].astype(str).str.strip().str.upper()
DESCRIPTION_NORMALIZED = {k.upper(): v for k, v in DESCRIPTION.items()}

# Filter rows containing compressor parameters
param_rows = df_raw[df_raw["DESCRIPTION"].isin(DESCRIPTION_NORMALIZED.keys())].copy()

# Map to short names
param_rows["Parameter"] = param_rows["DESCRIPTION"].map(DESCRIPTION_NORMALIZED)

hour_columns = df_raw.columns[7:31]  # assuming hourly columns are columns 7-30



In [9]:
records = []

for _, row in param_rows.iterrows():
    param_name = row["Parameter"]
    for i, col in enumerate(hour_columns):
        timestamp = datetime.strptime(LOG_DATE, "%Y-%m-%d") + timedelta(hours=i)
        value = row[col]
        records.append({"Timestamp": timestamp, "Parameter": param_name, "Value": value})

df_long = pd.DataFrame(records)
df_long.head(30)

,Timestamp,Parameter,Value
0,2025-07-01 00:00:00,Load,95.1
1,2025-07-01 01:00:00,Load,99.0
2,2025-07-01 02:00:00,Load,99.1
3,2025-07-01 03:00:00,Load,97.9
4,2025-07-01 04:00:00,Load,96.0
5,2025-07-01 05:00:00,Load,100.1
6,2025-07-01 06:00:00,Load,103.0
7,2025-07-01 07:00:00,Load,96.0
8,2025-07-01 08:00:00,Load,102.8
9,2025-07-01 09:00:00,Load,104.9


In [10]:
# Convert Value to numeric
df_long["Value"] = pd.to_numeric(df_long["Value"], errors='coerce')

df_ml = df_long.pivot_table(
    index="Timestamp",
    columns="Parameter",
    values="Value"
).reset_index()
print(df_long)


              Timestamp Parameter  Value
0   2025-07-01 00:00:00      Load   95.1
1   2025-07-01 01:00:00      Load   99.0
2   2025-07-01 02:00:00      Load   99.1
3   2025-07-01 03:00:00      Load   97.9
4   2025-07-01 04:00:00      Load   96.0
..                  ...       ...    ...
211 2025-07-01 19:00:00       VIB    NaN
212 2025-07-01 20:00:00       VIB    NaN
213 2025-07-01 21:00:00       VIB    NaN
214 2025-07-01 22:00:00       VIB    NaN
215 2025-07-01 23:00:00       VIB    NaN

[216 rows x 3 columns]


In [11]:
# Ensure all parameters appear in columns
for short_name in DESCRIPTION.values():
    if short_name not in df_ml.columns:
        df_ml[short_name] = pd.NA

# Optional: sort columns
df_ml = df_ml[['Timestamp'] + list(DESCRIPTION.values())]

print(df_ml.head())
df_ml.to_csv("GT1_Compressor_ML_Dataset.csv", index=False)


Parameter           Timestamp    CIP   CIT   CDP    CDT  IGV_Position  \
0         2025-07-01 00:00:00  127.4  24.0  10.0  346.0          76.0   
1         2025-07-01 01:00:00  117.4  23.0   9.0  664.0          62.0   
2         2025-07-01 02:00:00  125.4  23.0   9.0  662.0          61.0   
3         2025-07-01 03:00:00  135.8  23.0   8.0  629.0          61.0   
4         2025-07-01 04:00:00  133.4  24.0   8.0  327.0          57.0   

Parameter  Filter_DP  GT_Speed   VIB  Load  
0               23.0    3004.0  <NA>  95.1  
1               44.0    3018.0  <NA>  99.0  
2               55.0    3017.0  <NA>  99.1  
3               55.0    3014.0  <NA>  97.9  
4               66.0    3018.0  <NA>  96.0  


In [15]:
df_ml["presure_ratio"] = df_ml["CDP"]/df_ml["CIP"] # INDICATES THE COMPRESSOR HEALTH
df_ml["Temperature_ratio"] = df_ml["CDT"]/df_ml["CIT"] #INDICATES THERMODYNAMIC EFFICIENCY



KeyError: 'pressure_ratio'